In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# from main import get_vsa, get_similarity, get_test_data
# import matplotlib.pyplot as plt

# device = 'cuda'
# vsa = get_vsa(device)
# v1 = vsa.get_vector([(1,1,5,6),(2,0,2,5),(1,2,0,9)], quantize=True)
# print(get_similarity(v1, vsa.get_vector((2,0,2,3)), True))
# print(get_similarity(v1, vsa.get_vector((2,0,2,5)), True))
# dl = get_test_data(vsa)
# plt.imshow(dl.dataset[226][0])

In [1]:
from tools.model_quantization import *
from datasets import get_test_data
from models.nn_non_decomposed import MultiConceptNonDecomposed
import torch
import os.path

root = "/home/aifusenno1/multi-concept-MNIST/tests/HARDWARE-1024dim-256fd-3x-3y-7color/algo1"
pt = os.path.join(root, "model_weights_3objs_128batch_20epoch_300000samples_0.0485loss_10-14-21-53.pt")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MultiConceptNonDecomposed(dim=1024, device=device)
model.eval()
checkpoint = torch.load(pt, map_location=device)
model.load_state_dict(checkpoint)

dl = get_test_data(root, shuffle=True, num_samples=300, max_num_objects=3)
test(model, dl, max_iter=100)


test 1 obj 100 dataset exists, loading...
test 2 obj 100 dataset exists, loading...
test 3 obj 100 dataset exists, loading...
tensor([[-0.0054,  1.9841, -0.0191,  ..., -2.1270,  1.9476, -1.9726]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.,  2.,  0.,  ..., -2.,  2., -2.]])
0.9846702100026725
0.062209257519444325


In [2]:
merge_biases(model)
remove_biases(model)
test(model, dl, max_iter=100)

tensor([[ 0.9244, -0.9811,  0.9973,  ..., -1.1247,  1.0231,  0.9139]],
       grad_fn=<AddmmBackward0>)
tensor([[ 1., -1.,  1.,  ..., -1.,  1.,  1.]])
0.9819421201649279
0.06821500729206986


In [3]:
stop_profiling = register_activation_profiling_hooks(model)
test(model, dl, max_iter=100)
stop_profiling()

tensor([[ 1.8389,  1.9942, -0.1733,  ..., -2.0256, -0.1640, -0.1240]],
       grad_fn=<AddmmBackward0>)
tensor([[ 2.,  2.,  0.,  ..., -2.,  0.,  0.]])
0.9959002355537793
0.036803408260779956


In [4]:
quantize_layer_weights(model)
quantize_activations(model)
quantize_bottleneck_layers(model)
quantize_averaging_layer(model)
quantize_layer_biases(model)

In [5]:
test(model, dl, max_iter=100, quantized=True)

tensor([[ 0.4688, -0.3750, -0.0234,  ...,  1.3359, -2.1094, -2.2734]],
       grad_fn=<MulBackward0>)
tensor([[ 1.,  1., -1.,  ...,  1., -1., -1.]])
0.8127386517232598
0.8986776949155448


In [ ]:
for c in model.model.modules():
    print(c)